Exploratory Data Analysis (EDA)

This notebook explores the raw NordTech order dataset before any cleaning or transformation.
The goal is to understand the structure, quality, and issues in the data so we can design a robust cleaning pipeline in the next step.

We will:
- Load the raw dataset and validation dataset
- Inspect column types and missing values
- Identify inconsistent formats (dates, prices, regions, payment methods)
- Look for duplicates and logical errors
- Document all issues for the cleaning pipeline

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('../data/raw/nordtech_data.csv')
df.head()

,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,kundtyp,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg
0,ORD-2024-00001,ORD-2024-00001-1,2024-05-19,2024-05-22,SKU-WC001,Webbkamera HD,Tillbehör,1,SEK 799,Uppsala,Privat,Kort,KND-53648,Levererad,NaN,NaN,NaN
1,ORD-2024-00002,ORD-2024-00002-1,2024-12-02,5 december 2024,SKU-HB001,USB-C Hub 7-port,Tillbehör,1,549.00,Göteborg,Privat,Swish,KND-84095,Levererad,NaN,NaN,NaN
2,ORD-2024-00003,ORD-2024-00003-1,2024-12-31,2025-01-03,SKU-SD001,Extern SSD 1TB,Lagring,1,1199.00,NaN,Företag,Faktura,KND-91748,Levererad,Stämmer inte överens med produktbeskrivningen.,2025-01-12,2.0
3,ORD-2024-00003,ORD-2024-00003-2,2024-12-31,2025-01-03,SKU-SD002,Extern SSD 500GB,Lagring,10,699 kr,Stockholm,Företag,FAKTURA,KND-91748,Mottagen,"Leveransen tog lite längre än utlovat, men pro...",2025-01-14,3.0
4,ORD-2024-00003,ORD-2024-00003-3,2024-12-31,2025-01-03,SKU-MS001,Trådlös Mus X1,Tillbehör,1,399.00,Stockholm,Företag,Faktura,KND-91748,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2767 entries, 0 to 2766
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         2767 non-null   object 
 1   orderrad_id      2767 non-null   object 
 2   orderdatum       2767 non-null   object 
 3   leveransdatum    2767 non-null   object 
 4   produkt_sku      2767 non-null   object 
 5   produktnamn      2767 non-null   object 
 6   kategori         2767 non-null   object 
 7   antal            2767 non-null   object 
 8   pris_per_enhet   2767 non-null   object 
 9   region           2612 non-null   object 
 10  kundtyp          2767 non-null   object 
 11  betalmetod       2651 non-null   object 
 12  kund_id          2767 non-null   object 
 13  leveransstatus   2673 non-null   object 
 14  recension_text   1355 non-null   object 
 15  recensionsdatum  1355 non-null   object 
 16  betyg            1355 non-null   float64
dtypes: float64(1),

What to check:

- Which columns are objects but should be dates or numbers?

- How many missing values?

In [ ]:
df.isna().sum()

order_id              0
orderrad_id           0
orderdatum            0
leveransdatum         0
produkt_sku           0
produktnamn           0
kategori              0
antal                 0
pris_per_enhet        0
region              155
kundtyp               0
betalmetod          116
kund_id               0
leveransstatus       94
recension_text     1412
recensionsdatum    1412
betyg              1412
dtype: int64

What to check:

Which columns have null?

In [ ]:
date_cols = ["orderdatum", "leveransdatum", "recensionsdatum"]

for col in date_cols:
    print(f"\nUnique samples from {col}:")
    print(df[col].dropna().astype(str).unique()[:20])


Unique samples from orderdatum:
['2024-05-19' '2024-12-02' '2024-12-31' '2024-04-22' '2024-07-01'
 '2024-03-10' '2024-06-16' '2024-08-07' '2024-06-10' '2024/06/10'
 '2024-10-26' '2024-07-26' '2024-04-08' '2024-02-23' '2024-05-14'
 '2024-11-25' '2024-04-27' '2024-10-24' '2024-12-05' '2024-06-20']

Unique samples from leveransdatum:
['2024-05-22' '5 december 2024' '2025-01-03' '2024-04-26' '2024-07-05'
 '2024-03-12' '2024-06-20' '2024-08-14' '2024-06-12' '2024-10-30'
 '2024-07-28' '2024-04-11' '2024-02-25' '2024-05-16' '2024-11-27'
 '2024-05-01' '2024/10/27' '2024-12-07' '2024-06-22' '2024-11-02']

Unique samples from recensionsdatum:
['2025-01-12' '2025-01-14' '2024-07-12' '2024-03-19' '2024-08-24'
 '2024-08-21' '2024-06-15' '2024-05-23' '7 maj 2024' '2024-12-08'
 '2024-12-16' '2024-07-06' '2024-07-21' '2024-12-09' '2024-03-17'
 '2024-09-26' '2024-11-08' '2024-02-15' '2024-02-05' '2024-08-30']


What to check:

- Multiple date formats

In [ ]:
def clean_date(df):
   
    df_cleaned = df.copy()

    date_cols = [
        "orderdatum",
        "leveransdatum",
        "recensionsdatum"
    ]

    for col in date_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = pd.to_datetime(
                df_cleaned[col],
                errors="coerce"
            )

    return df_cleaned


In [ ]:
df_cleaned_date = clean_date(df)
df_cleaned_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2767 entries, 0 to 2766
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         2767 non-null   object        
 1   orderrad_id      2767 non-null   object        
 2   orderdatum       2580 non-null   datetime64[ns]
 3   leveransdatum    2575 non-null   datetime64[ns]
 4   produkt_sku      2767 non-null   object        
 5   produktnamn      2767 non-null   object        
 6   kategori         2767 non-null   object        
 7   antal            2767 non-null   object        
 8   pris_per_enhet   2767 non-null   object        
 9   region           2612 non-null   object        
 10  kundtyp          2767 non-null   object        
 11  betalmetod       2651 non-null   object        
 12  kund_id          2767 non-null   object        
 13  leveransstatus   2673 non-null   object        
 14  recension_text   1355 non-null   object 

In [ ]:
df["pris_per_enhet"].astype(str).unique()[:30]

array(['SEK 799', '549.00', '1199.00', '699 kr', '399.00', '799.00',
       '1899.00', '599.00', '4999.00', '5999.00', '18999.00', '1299.00',
       '14999.00', '699.00', '2499.00', '7999.00', '499.00', '4999 kr',
       '599 kr', '899.00', '899 kr', '799:-', '18999 kr', '1 899,00',
       '7999 kr', '399 kr', '599:-', 'SEK 1899', '549 kr', '1299:-'],
      dtype=object)

What to check:
'SEK 799'  '699 kr'  '799:-'   '1 899,00'

In [8]:
def clean_prices(df):
    df_cleaned=df.copy()
    if "pris_per_enhet" not in df_cleaned.columns:
        return df_cleaned
    df_cleaned["pris_per_enhet"] = (
        df_cleaned["pris_per_enhet"]
        .astype(str)
        .str.replace(" ", "")
        .str.replace("SEK", "")
        .str.replace("kr", "")
        .str.replace(":-", "")
        .str.replace(",", ".") )
    df_cleaned["pris_per_enhet"] = pd.to_numeric(df_cleaned["pris_per_enhet"], errors="coerce")
    return df_cleaned

df_clean_prices = clean_prices(df)
df_clean_prices["pris_per_enhet"].unique()

array([  799.,   549.,  1199.,   699.,   399.,  1899.,   599.,  4999.,
        5999., 18999.,  1299., 14999.,  2499.,  7999.,   499.,   899.])

In [ ]:
df["region"].unique()

array(['Uppsala', 'Göteborg', nan, 'Stockholm', 'Örebro', 'örebro',
       'Orebro', 'Norrland', 'Linköping', 'Malmö', 'Västerås', 'GÖTEBORG',
       'LINKÖPING', 'stockholm', 'Gothenburg', 'STHLM', 'STOCKHOLM',
       'malmo', 'linköping', 'uppsala', 'Sthlm', 'GBGB', 'västerås',
       'MALMÖ', 'Sthml', 'Gbg', 'NORRLAND', 'ÖREBRO', 'norrland', 'Norr',
       'Vasteras', 'UPPSALA', 'göteborg', 'Malmo', 'Linkoping',
       'VÄSTERÅS', 'malmö'], dtype=object)

What to check:
'Orebro'   'STHLM'   'GBGB'  'malmo'   'Sthml'   'Gbg'  
missing value 

In [36]:
def clean_region(df):
    df_cleaned=df.copy()
    mapping = {
        "sthlm": "stockholm",
        "sthml": "stockholm",
        "gothenburg": "göteborg",
        "gbgb": "göteborg",
        "gbg": "göteborg",
        "linkoping": "linköping",
        "malmo": "malmö",
        "orebro": "örebro",
        "vasteras": "västerås",
        "norr": "norrland",
        }
    df_cleaned["region"] = ( df_cleaned["region"] .astype(str) .str.strip() .str.lower() .replace(mapping) )
    return df_cleaned

df_clean_region = clean_region(df)
df_clean_region["region"].unique()

array(['uppsala', 'göteborg', 'nan', 'stockholm', 'örebro', 'norrland',
       'linköping', 'malmö', 'västerås'], dtype=object)

In [ ]:
df["betalmetod"].unique()

array(['Kort', 'Swish', 'Faktura', 'FAKTURA', 'Mobilbetalning', 'faktura',
       'KORT', 'SWISH', 'Kreditkort', 'swish', nan, 'Invoice', 'Visa',
       'kort', 'Mastercard'], dtype=object)

What to check:
- Kort, kort, KORT
- Visa, Kreditkort
- Swish, swish, SWISH
- Faktura, faktura, FAKTURA
- Missing values

In [ ]:
df["leveransstatus"].unique()

array(['Levererad', 'Mottagen', nan, 'LEVERERAD', 'Retur', 'retur',
       'Under transport', 'Skickad', 'Returnerad', 'levererad',
       'Återsänd', 'På väg', 'under transport', 'RETUR',
       'UNDER TRANSPORT'], dtype=object)

What to check:
- Levererad, LEVERERAD
- Retur, retur, RETUR, Återsänd, Returnerad
- Under transport, under transport, UNDER TRANSPORT
- Missing values

In [ ]:
df.duplicated().sum()

np.int64(67)

Check for Duplicates

In [ ]:
# Delivery earlier than order
mask = pd.to_datetime(df["leveransdatum"], errors="coerce") < pd.to_datetime(df["orderdatum"], errors="coerce")
df[mask].head()

,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,kundtyp,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg
54,ORD-2024-00037,ORD-2024-00037-1,2024-02-17,2024-02-14,SKU-SD002,Extern SSD 500GB,Lagring,1,699.00,Göteborg,Privat,Kreditkort,KND-35320,Retur,NaN,NaN,NaN
59,ORD-2024-00038,ORD-2024-00038-5,2024-11-01,2024-10-31,SKU-LP002,Laptop Budget 14,Datorer,2,7999.00,Göteborg,Företag,Faktura,KND-76077,Levererad,NaN,NaN,NaN
68,ORD-2024-00044,ORD-2024-00044-2,2024-10-15,2024-10-10,SKU-WC001,Webbkamera HD,Tillbehör,1,799.00,Linköping,Privat,Faktura,KND-75717,Levererad,NaN,NaN,NaN
112,ORD-2024-00072,ORD-2024-00072-2,2024-04-13,2024-04-11,SKU-MS002,Ergonomisk Mus Pro,Tillbehör,1,699.00,Uppsala,Konsument,Kort,KND-81573,Levererad,NaN,NaN,NaN
143,ORD-2024-00093,ORD-2024-00093-1,2024-04-26,2024-04-22,SKU-KB001,Mekaniskt Tangentbord K7,Tillbehör,2,1299.00,MALMÖ,Företag,Kort,KND-29299,Levererad,Mycket nöjd med köpet. Snabb leverans också!,13/05/2024,5.0
